In [2]:
from MCTSOpt import Tree
from MCTSOpt import SelectionRule_Stocastic
from MCTSOpt.ParameterOpt import LogisticSearch
from time import time

In [11]:
from Model import getexploremodel
model = getexploremodel()

nparameters = 0
for layer in model.trainable_weights:
    print(layer.shape)
    nparameters += layer.shape.num_elements()
print("Number of parameters: ", nparameters)



(1, 32)
(32,)
(32, 64)
(64,)
(64, 64)
(64,)
(64, 32)
(32,)
(32, 1)
(1,)
Number of parameters:  8449


/usr/local/lib/python3.10/dist-packages/keras_core/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from ScoringFunction import MCTS_MacroScore
objfunc = MCTS_MacroScore(model, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)

In [ ]:
depthscale = [1.0, 0.7, 0.5, 0.3, 0.2, 0.1]
depthlimit = len(depthscale)+5



indata = LogisticSearch(startset, lbounds, ubounds, depthscale=depthscale)
indata.setevaluator(wrappedfunction)

In [ ]:




#---Tree Main Run loop---
#Critical Parameters to Set
tree = Tree(seeddata=indata, 
        playouts=20, 
        selectfunction=SelectionRule_Stocastic, 
        headexpansion=10)
tree.expand(nExpansions=1)
tree.setconstant(5.37e2)
starttime = time()
for iLoop in range(1,50):
    print("Loop Number: %s"%(iLoop))
    tree.playexpand(nExpansions=1, depthlimit=depthlimit)
    tree.autoscaleconstant(scaleboost=2.0)
    tree.simulate(nSimulations=1)
    curmin = wrappedfunction.bestscore
    if fabs(curmin - f_target) < 1e-4 and np.linalg.norm(wrappedfunction.best_x - testfunction.x_best) < 1e-4:
        break
    curtime = time()
    print("Search Duration: %s, Best Score:%s"%(curtime-starttime, curmin))
bestscore = wrappedfunction.bestscore   
evalcount = wrappedfunction.evalcount 
best_x = wrappedfunction.best_x
return bestscore, best_x, evalcount